<a href="https://www.kaggle.com/code/crypticghost/4-11-lightgbm-optuna?scriptVersionId=204767995" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e11/sample_submission.csv
/kaggle/input/playground-series-s4e11/train.csv
/kaggle/input/playground-series-s4e11/test.csv


In [2]:
import warnings
import lightgbm as lgb
from sklearn.metrics import accuracy_score

## Importing the dataset

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s4e11/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e11/test.csv')
sample = pd.read_csv('/kaggle/input/playground-series-s4e11/sample_submission.csv')

In [4]:
train.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


In [5]:
test.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,140700,Shivam,Male,53.0,Visakhapatnam,Working Professional,Judge,NaN,2.0,NaN,NaN,5.0,Less than 5 hours,Moderate,LLB,No,9.0,3.0,Yes
1,140701,Sanya,Female,58.0,Kolkata,Working Professional,Educational Consultant,NaN,2.0,NaN,NaN,4.0,Less than 5 hours,Moderate,B.Ed,No,6.0,4.0,No
2,140702,Yash,Male,53.0,Jaipur,Working Professional,Teacher,NaN,4.0,NaN,NaN,1.0,7-8 hours,Moderate,B.Arch,Yes,12.0,4.0,No
3,140703,Nalini,Female,23.0,Rajkot,Student,NaN,5.0,NaN,6.84,1.0,NaN,More than 8 hours,Moderate,BSc,Yes,10.0,4.0,No
4,140704,Shaurya,Male,47.0,Kalyan,Working Professional,Teacher,NaN,5.0,NaN,NaN,5.0,7-8 hours,Moderate,BCA,Yes,3.0,4.0,No


## Check for missing values

In [6]:
numeric_columns = train.select_dtypes(['number'])
categorical_columns = train.select_dtypes(['object'])

In [7]:
# median values in each numerical column
for col in numeric_columns:
    print(col, '->', train[col].median())

id -> 70349.5
Age -> 42.0
Academic Pressure -> 3.0
Work Pressure -> 3.0
CGPA -> 7.77
Study Satisfaction -> 3.0
Job Satisfaction -> 3.0
Work/Study Hours -> 6.0
Financial Stress -> 3.0
Depression -> 0.0


In [8]:
# Unique values for each categorical column
for col in categorical_columns:
    if col != 'Name':
        print(col, '->', train[col].isna().sum())

Gender -> 0
City -> 0
Working Professional or Student -> 0
Profession -> 36630
Sleep Duration -> 0
Dietary Habits -> 4
Degree -> 2
Have you ever had suicidal thoughts ? -> 0
Family History of Mental Illness -> 0


In [9]:
# Filling num. columns with missing values

for col in numeric_columns:
    if col != 'Depression':
        train[col] = train[col].fillna(train[col].median())
        test[col] = test[col].fillna(train[col].median())

In [10]:
# Fill unknown in categorical columns

for col in categorical_columns:
    train[col] = train[col].fillna('Unknown')
    test[col] = test[col].fillna('Unknown')

## Normalization of numerical variables

In [11]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

for col in numeric_columns:
    if col != 'Depression' and col != 'id':
        train[col] = ss.fit_transform(train[[col]])
        test[col] = ss.transform(test[[col]])

## Label Encoding of categorical variables

In [12]:
from sklearn.preprocessing import LabelEncoder

for col in categorical_columns:
    # Fit LabelEncoder on train data
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])

    # Set categories in test to match train, then transform
    test[col] = pd.Categorical(test[col], categories=le.classes_)
    test[col] = test[col].cat.codes

In [13]:
train.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,11,0,0.695360,50,1,10,-0.045698,1.589714,0.033784,0.018013,-0.772518,29,7,33,0,-1.363057,-0.699617,0,0
1,1,407,1,-1.161867,93,1,55,-0.045698,0.795176,0.033784,0.018013,0.016183,27,20,63,1,0.193928,0.007793,0,1
2,2,417,1,-0.596624,97,0,59,3.194278,0.000638,1.869755,-1.632006,0.016183,15,7,21,1,-0.844062,-1.407027,0,1
3,3,417,1,-1.484863,64,1,55,-0.045698,1.589714,0.033784,0.018013,-1.561219,27,15,28,1,0.972421,-1.407027,1,1
4,4,286,0,-0.838871,37,1,9,-0.045698,-1.588437,0.033784,0.018013,-1.561219,15,20,28,1,0.712923,0.715203,1,0


In [14]:
test.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,140700,336,1,1.018356,97,1,31,-0.045698,-0.793900,0.033784,0.018013,1.593586,27,15,63,0,0.712923,0.007793,1
1,140701,324,0,1.422101,44,1,21,-0.045698,-0.793900,0.033784,0.018013,0.804884,27,15,20,0,-0.065570,0.715203,0
2,140702,413,1,1.018356,33,1,55,-0.045698,0.795176,0.033784,0.018013,-1.561219,19,15,17,1,1.491416,0.715203,0
3,140703,217,0,-1.404114,78,0,59,3.194278,0.000638,-1.389094,-3.282026,0.016183,29,15,36,1,0.972421,0.715203,0
4,140704,332,1,0.533862,36,1,55,-0.045698,1.589714,0.033784,0.018013,1.593586,19,15,29,1,-0.844062,0.715203,0


## Hyper parameter tuning

In [15]:
X_train = train.iloc[:, 1:-1]
y_train = train.iloc[:, -1]

X_test = test.iloc[:, 1:]

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
x1, x2, y1, y2 = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [18]:
import optuna

{'learning_rate': 0.2801762685198732, 'max_depth': 4, 'min_child_samples': 167, 'subsample': 0.9623377690679852, 'colsample_bytree': 0.6886815351218578, 'reg_alpha': 0.0001584697686817359, 'reg_lambda': 4.460876544311563e-06, 'num_leaves': 16, 'min_gain_to_split': 0.35743226335833506}
Best accuracy score:  0.9409026297085998

In [19]:
# def objective(trial):
#     warnings.filterwarnings("ignore", category=UserWarning, module='lightgbm')
    
#     param = {
#     'device': 'gpu',
#     'objective': 'binary',
#     'metric': 'binary_error',  
#     'learning_rate': trial.suggest_float('learning_rate', 0.2, 0.3),  
#     'max_depth': trial.suggest_int('max_depth', 3, 4),
#     'min_child_samples': trial.suggest_int('min_child_samples', 167, 180),  
#     'subsample': trial.suggest_float('subsample', 0.96, 1.0),  
#     'colsample_bytree': trial.suggest_float('colsample_bytree', 0.68, 1.0),  
#     'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 1e-2, log=True),  
#     'reg_lambda': trial.suggest_float('reg_lambda', 4.46e-6, 1e-2, log=True),  
#     'num_leaves': trial.suggest_int('num_leaves', 16, 20),  
#     'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.35, 0.5),  
# }
    
#     model = lgb.LGBMClassifier(**param)
#     model.fit(
#         x1, 
#         y1, 
#         eval_set=[(x2, y2)],
#         eval_metric='binary_error',  
#         callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
#     )
    
#     # Get predictions and calculate accuracy score
#     y_pred = model.predict(x2)
#     accuracy = accuracy_score(y2, y_pred)
    
#     return accuracy

In [20]:
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=200)

# print("Best hyperparameters: ", study.best_params)
# print("Best accuracy score: ", study.best_value)

In [21]:
params = {'learning_rate': 0.2462939387796175, 'max_depth': 4, 'min_child_samples': 169, 'subsample': 0.9918787184984436, 'colsample_bytree': 0.9812679411972848, 'reg_alpha': 0.0001612689162270162, 'reg_lambda': 1.661164830402983e-05, 'num_leaves': 18, 'min_gain_to_split': 0.4411581546676269}

In [22]:
model = lgb.LGBMClassifier(**params)

model.fit(
        X_train, 
        y_train, 
        eval_set=[(X_train, y_train)],
        eval_metric='binary_error',  
        callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
    )
    
# Get predictions and calculate accuracy score
y_pred = model.predict(X_test)

[LightGBM] [Warning] min_gain_to_split is set=0.4411581546676269, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.4411581546676269
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_gain_to_split is set=0.4411581546676269, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.4411581546676269
[LightGBM] [Info] Number of positive: 25567, number of negative: 115133
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014625 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 664
[LightGBM] [Info] Number of data points in the train set: 140700, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.181713 -> initscore=-1.504786
[LightGBM] [Info] Start training from score -1.504786
[LightGBM] [Warning] No further splits with positive 

In [23]:
sample['Depression'] = y_pred
sample.to_csv('submission.csv', index=False)